In [0]:
dbutils.widgets.text(
  "statement",
  "select * from system.access.workspaces_latest where workspace_id = dataops_prd.libs.get_workspace_id()"
)
statement = dbutils.widgets.get("statement")
warehouse_id = dbutils.widgets.get("warehouse_id")
if not statement:
  raise Exception("statement is required")

In [0]:
def execute_sql(statement,warehouse_id="29c4dccd4d8d7e90"):
  import requests
  import json
  import time

  token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
  headers = {
    "Authorization" : f"Bearer {token}",
    "Content-Type" : "application/json"
    }
  endpoint = '/api/2.0/sql/statements'
  query = "select workspace_url from dataops_prd.libs.current_workspace"
  workspace_url = spark.sql(query).first()[0]
  url = f"https://{workspace_url}{endpoint}"

  payload_update = {"statement": statement,
                    "wait_timeout": "50s",
                    "warehouse_id": warehouse_id,
                    "on_wait_timeout": "CONTINUE",
                    "disposition" : "INLINE",
                    "format": "JSON"
                    }
  struct_body = json.dumps(payload_update)
  r = requests.post(url, headers=headers, data=struct_body)
  if r.json()["status"]["state"] in ("PENDING","RUNNING"):
    statement_id = r.json()["statement_id"]
    url = f"https://{workspace_url}/api/2.0/sql/statements/{statement_id}"
    while r.json()["status"]["state"] in ("PENDING","RUNNING"):
      r = requests.get(url, headers=headers)
      print(r.json()["status"]["state"])
      time.sleep(60)
  return r.json()

In [0]:
execute_sql(statement=statement)